In [11]:
output_itemid = [40048,40049,40051,40054,40067,40085,40094,40286,40294,226559,226560,226561,226571,226573,226575,226576,226579,226580,226582,226583,226588,226589,226599,226626,226627,226633,227510,227511,227701]
output_itemid_list = '40048,40049,40051,40054,40067,40085,40094,40286,40294,226559,226560,226561,226571,226573,226575,226576,226579,226580,226582,226583,226588,226589,226599,226626,226627,226633,227510,227511,227701'
input_itemid =[220864,220949,220970,221385,221456,221668,221749,221794,221828,221906,222011,222056,223258,223260,223262,225152,225154,225166,225168,225799,225823,225828,225834,225943,225944,225974,226089,226364,226372,226452,226453,227522,227523]
input_itemid_list = '220864,220949,220970,221385,221456,221668,221749,221794,221828,221906,222011,222056,223258,223260,223262,225152,225154,225166,225168,225799,225823,225828,225834,225943,225944,225974,226089,226364,226372,226452,226453,227522,227523'
lab_itemid = [51221,51301,51265,51222,51249,51248,51250,51279,51277,50971,50983,50902,50882,50868,51006,50912,50931,50960,50893,50970,51237,51274,51275,51244,51254,51256,51146,51200,50885,50820,50802,50804,50821,50818,51491,51498,50813,50861,50878,50863,50862]
lab_itemid_list = '51221,51301,51265,51222,51249,51248,51250,51279,51277,50971,50983,50902,50882,50868,51006,50912,50931,50960,50893,50970,51237,51274,51275,51244,51254,51256,51146,51200,50885,50820,50802,50804,50821,50818,51491,51498,50813,50861,50878,50863,50862'
prescript_itemid = ['NACLFLUSH','ACET325','HEPA5I','INSULIN','DOCU100','D5W250','SENN187','NS1000','NS500','FURO40I','NS100','VANC1F','VANCOBASE','BISA10R','PANT40','MAG2PM','NS250','METO25','KCL20PM','KCLBASE2','PROP100IG','MICROK10','KCL20P','PANT40I','DOCU100L','MORP2I','VIAL','METO5I','LR1000','DEX50SY','CALG1I','ALBU3H','LORA2I','ONDAN4I','KCL40I','PERC','FRBD50','MAGS1I','BISA5','ASA81','PHEN10I','METO50','IPRA2H','INHRIV','ACET650R','CHLO15L','HEPBASE','ASA325','FENT2I','SW50']
prescript_itemid_list = "'NACLFLUSH','ACET325','HEPA5I','INSULIN','DOCU100','D5W250','SENN187','NS1000','NS500','FURO40I','NS100','VANC1F','VANCOBASE','BISA10R','PANT40','MAG2PM','NS250','METO25','KCL20PM','KCLBASE2','PROP100IG','MICROK10','KCL20P','PANT40I','DOCU100L','MORP2I','VIAL','METO5I','LR1000','DEX50SY','CALG1I','ALBU3H','LORA2I','ONDAN4I','KCL40I','PERC','FRBD50','MAGS1I','BISA5','ASA81','PHEN10I','METO50','IPRA2H','INHRIV','ACET650R','CHLO15L','HEPBASE','ASA325','FENT2I','SW50'"

NEW_ADMISSION_IDS = []
files = ['list_17439_remove_mismatch.txt']
for file in files:
    f = open('/home/kanit/mimic3/'+file, 'r')
    adms = f.readlines();
    for adm in adms:
        NEW_ADMISSION_IDS.append(int(adm))
ADMISSION_IDS = NEW_ADMISSION_IDS
print len(ADMISSION_IDS)

# just set 'ADMISSION_IDS' for code to work

17439


In [12]:
# DB connection:
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
try:
    # conn = psycopg2.connect("dbname='mimic' user='kanit' host='melady1.usc.edu' password='abcd6712'")
    conn = psycopg2.connect("dbname='mimic' user='kanit' host='localhost' password='abcd6712'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


In [ ]:
ICD9 = []
ct =0 ;
cate20=0
for aid in ADMISSION_IDS:
    print str(ct) + " : " + str(aid)
    ct+=1
    cur = conn.cursor()
    cur.execute('SELECT icd9_code FROM mimiciii.DIAGNOSES_ICD WHERE hadm_id='+str(aid)+' ORDER BY seq_num')
    icd9s = cur.fetchall() 
    list_icd9 = []
    for icd9 in icd9s:
        icd = icd9[0];
        
        if(icd[0] =='V'):
            label_name = 19
            numstr = icd[0:3]+'.'+icd[3:len(icd)]
        elif(icd[0] =='E'):
            cate20 +=1
            label_name = 20
            numstr = icd
        else:
            num = float(icd[0:3])
            numstr = icd[0:3]+'.'+icd[3:len(icd)]
            if(num >= 1 and num <= 139 ) :
                label_name = 0
            if(num >= 140 and num <= 239 ) :
                label_name = 1
            if(num >= 240 and num <= 279 ) :
                label_name = 2
            if(num >= 280 and num <= 289 ) :
                label_name = 3
            if(num >= 290 and num <= 319 ) :
                label_name = 4
            if(num >= 320 and num <= 389 ) :
                label_name = 5
            if(num >= 390 and num <= 459 ) :
                label_name = 6
            if(num >= 460 and num <= 519 ) :
                label_name = 7
            if(num >= 520 and num <= 579 ) :
                label_name = 8
            if(num >= 580 and num <= 629 ) :
                label_name = 9
            if(num >= 630 and num <= 677 ) :
                label_name = 10
            if(num >= 680 and num <= 709 ) :
                label_name = 11
            if(num >= 710 and num <= 739 ) :
                label_name = 12
            if(num >= 740 and num <= 759 ) :
                label_name = 13
            if(num >= 760 and num <= 779 ) :
                label_name = 14
            if(num >= 780 and num <= 789 ) :
                label_name = 15
            if(num >= 790 and num <= 796 ) :
                label_name = 16
            if(num >= 797 and num <= 799 ) :
                label_name = 17
            if(num >= 800 and num <= 999 ) :
                label_name = 18
        list_icd9.append([aid,icd,numstr,label_name]);
    ICD9.append(list_icd9)

0 : 153566
1 : 131917
2 : 164444
3 : 131359
4 : 174959
5 : 191210
6 : 152392
7 : 191148
8 : 197882
9 : 140212
10 : 168184
11 : 191928
12 : 149609
13 : 113210
14 : 162189
15 : 144497
16 : 194406
17 : 131870
18 : 125646
19 : 170691
20 : 197979
21 : 179949
22 : 136319
23 : 102738
24 : 188488
25 : 156416
26 : 174399
27 : 117108
28 : 122781
29 : 178985
30 : 137515
31 : 135053
32 : 136339
33 : 170590
34 : 161860
35 : 130978
36 : 183487
37 : 150897
38 : 101853
39 : 113198
40 : 162296
41 : 145218
42 : 196485
43 : 173407
44 : 187989
45 : 193878
46 : 120113
47 : 121138
48 : 166576
49 : 186894
50 : 107019
51 : 175538
52 : 101424
53 : 125757
54 : 191665
55 : 178138
56 : 112235
57 : 197042
58 : 102309
59 : 195762
60 : 151432
61 : 151200
62 : 158793
63 : 120972
64 : 116111
65 : 192742
66 : 160506
67 : 123701
68 : 110731
69 : 189308
70 : 170288
71 : 120925
72 : 170734
73 : 145127
74 : 109679
75 : 194032
76 : 109695
77 : 137417
78 : 103674
79 : 138450
80 : 181443
81 : 114622
82 : 181893
83 : 115231
84

In [4]:
# save it
import numpy as np;
np.save('ICD9-17000',ICD9)

In [10]:
ICD9[0]

[[153566, '41519', '415.19', 6],
 [153566, '56722', '567.22', 8],
 [153566, '70703', '707.03', 11],
 [153566, '00845', '008.45', 0],
 [153566, '591', '591.', 9],
 [153566, '5119', '511.9', 7],
 [153566, '45341', '453.41', 6],
 [153566, '7907', '790.7', 16],
 [153566, '5789', '578.9', 8],
 [153566, '5778', '577.8', 8],
 [153566, '04104', '041.04', 0],
 [153566, '04149', '041.49', 0],
 [153566, '0417', '041.7', 0],
 [153566, '53081', '530.81', 8],
 [153566, '2724', '272.4', 2],
 [153566, '60000', '600.00', 9],
 [153566, '2811', '281.1', 3],
 [153566, '4019', '401.9', 6],
 [153566, '5533', '553.3', 8],
 [153566, '56210', '562.10', 8],
 [153566, '42789', '427.89', 6],
 [153566, '70722', '707.22', 11],
 [153566, '36250', '362.50', 5],
 [153566, 'V1253', 'V12.53', 19],
 [153566, 'V431', 'V43.1', 19],
 [153566, 'V4561', 'V45.61', 19],
 [153566, 'E9342', 'E9342', 20],
 [153566, 'V4589', 'V45.89', 19],
 [153566, 'V1301', 'V13.01', 19],
 [153566, '79092', '790.92', 16]]